In [28]:
!pip install -q -U google-generativeai langchain langchain-community langchain-google-genai chromadb sentence-transformers tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


In [38]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [40]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

In [41]:
# --- Agent 1: Startup Type Interpreter Agent ---
def interpret_startup(startup_info):
    return {
        "type": startup_info["type"],
        "stage": startup_info["stage"],
        "relevant_metric_focus": "retention and engagement" if startup_info["type"].lower() == "saas" else "GMV"
    }

In [42]:
# --- Test Agent 1 ---
test_startups = [
    {"type": "SaaS", "stage": "Early"},
    {"type": "Marketplace", "stage": "Growth"},
    {"type": "D2C", "stage": "Seed"},
    {"type": "Subscription", "stage": "Scale"},
]

# --- Run and Print Outputs ---
for i, startup in enumerate(test_startups, 1):
    result = interpret_startup(startup)
    print(f"Test Case {i}:")
    print(f"Input: {startup}")
    print(f"Output: {result}\\n")

Test Case 1:
Input: {'type': 'SaaS', 'stage': 'Early'}
Output: {'type': 'SaaS', 'stage': 'Early', 'relevant_metric_focus': 'retention and engagement'}\n
Test Case 2:
Input: {'type': 'Marketplace', 'stage': 'Growth'}
Output: {'type': 'Marketplace', 'stage': 'Growth', 'relevant_metric_focus': 'GMV'}\n
Test Case 3:
Input: {'type': 'D2C', 'stage': 'Seed'}
Output: {'type': 'D2C', 'stage': 'Seed', 'relevant_metric_focus': 'GMV'}\n
Test Case 4:
Input: {'type': 'Subscription', 'stage': 'Scale'}
Output: {'type': 'Subscription', 'stage': 'Scale', 'relevant_metric_focus': 'GMV'}\n


In [43]:
# --- Agent 2: KPI Critique Agent ---
def critique_kpis_with_llm(kpis, context):
    prompt = f"""
You are a KPI analysis expert for startups.

Startup Type: {context['type']}
Startup Stage: {context['stage']}

Classify each of the following KPIs as a 'Vanity Metric' or an 'Impact Metric'.
Provide a one-line explanation for each.

KPIs: {', '.join(kpis)}
    """
    return llm.invoke(prompt).content


In [44]:
# --- Test Agent 2 ---
test_kpi_inputs = [
    {
        "kpis": ["Instagram Followers", "Website Visits", "Monthly Recurring Revenue", "Trial Conversion Rate"],
        "context": {"type": "SaaS", "stage": "Early"}
    },
    {
        "kpis": ["GMV", "Likes", "Conversion Rate", "Active Sellers"],
        "context": {"type": "Marketplace", "stage": "Growth"}
    },
    {
        "kpis": ["Impressions", "Repeat Purchase Rate", "Net Promoter Score"],
        "context": {"type": "D2C", "stage": "Seed"}
    },
    {
        "kpis": ["Subscribers", "Website Visits", "Churn Rate", "Engagement Rate"],
        "context": {"type": "Subscription", "stage": "Scale"}
    }
]

# --- Run and Print Outputs ---
for i, test_case in enumerate(test_kpi_inputs, 1):
    kpis = test_case["kpis"]
    context = test_case["context"]
    result = critique_kpis(kpis, context)
    print(f"Test Case {i}:")
    print(f"Input KPIs: {kpis}")
    print(f"Startup Context: {context}")
    print("Critique:")
    for kpi, status in result.items():
        print(f"- {kpi}: {status}")
    print()

Test Case 1:
Input KPIs: ['Instagram Followers', 'Website Visits', 'Monthly Recurring Revenue', 'Trial Conversion Rate']
Startup Context: {'type': 'SaaS', 'stage': 'Early'}
Critique:
- Instagram Followers: ❌ Vanity metric
- Website Visits: ❌ Vanity metric
- Monthly Recurring Revenue: ✅ Potential impact metric
- Trial Conversion Rate: ✅ Potential impact metric

Test Case 2:
Input KPIs: ['GMV', 'Likes', 'Conversion Rate', 'Active Sellers']
Startup Context: {'type': 'Marketplace', 'stage': 'Growth'}
Critique:
- GMV: ✅ Potential impact metric
- Likes: ❌ Vanity metric
- Conversion Rate: ✅ Potential impact metric
- Active Sellers: ✅ Potential impact metric

Test Case 3:
Input KPIs: ['Impressions', 'Repeat Purchase Rate', 'Net Promoter Score']
Startup Context: {'type': 'D2C', 'stage': 'Seed'}
Critique:
- Impressions: ❌ Vanity metric
- Repeat Purchase Rate: ✅ Potential impact metric
- Net Promoter Score: ✅ Potential impact metric

Test Case 4:
Input KPIs: ['Subscribers', 'Website Visits', 'Chu